In [1]:
### Binary classification -> Neural networks.
###classifying circles

In [2]:
##dependencies
from sklearn.datasets import make_circles


In [ ]:
## make   1000 samples 
n_samples = 1000
X, y = make_circles(
    n_samples,
    noise=0.02,
    random_state=30
)